In [ ]:
import pandas as pd
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import sys
from collections import Counter
import os

# PyABSA 라이브러리 임포트 (Quickstart 예시 기반)
from pyabsa import AspectTermExtraction as ATEPC, available_checkpoints # available_checkpoints도 함께 임포트합니다.

# MAPE를 위한 유틸리티 함수 (0으로 나누는 오류 방지)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_true = y_true != 0
    if np.sum(non_zero_true) == 0:
        return np.nan # 모든 y_true가 0인 경우 NaN 반환
    return np.mean(np.abs((y_true[non_zero_true] - y_pred[non_zero_true]) / y_true[non_zero_true])) * 100

# 1단계: 데이터 로드 및 전처리
print("--- 1단계: 데이터 로드 및 전처리 시작 ---")
try:
    data = []
    with open('review.json', 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    print(f"'{len(df)}'개의 리뷰 데이터를 성공적으로 로드했습니다.")
except FileNotFoundError:
    print("오류: 'review.json' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    sys.exit()

user_encoder = LabelEncoder()
business_encoder = LabelEncoder()
df['user_idx'] = user_encoder.fit_transform(df['user_id'])
df['business_idx'] = business_encoder.fit_transform(df['business_id'])
num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)
print(f"'{num_users}'명의 고유 사용자와 '{num_businesses}'개의 고유 레스토랑을 매핑했습니다.")

df['date'] = pd.to_datetime(df['date'], unit='ms')
df = df.sort_values(by='date').reset_index(drop=True)
total_reviews = len(df)
train_split = int(total_reviews * 0.7)
val_split = int(total_reviews * (0.7 + 0.1))
train_df = df.iloc[:train_split].copy()
val_df = df.iloc[train_split:val_split].copy()
test_df = df.iloc[val_split:].copy()
print(f"학습 세트 크기: {len(train_df)}개 리뷰")
print(f"검증 세트 크기: {len(val_df)}개 리뷰")
print(f"테스트 세트 크기: {len(test_df)}개 리뷰")

class ReviewDataset(Dataset):
    def __init__(self, df):
        self.user_indices = torch.tensor(df['user_idx'].values, dtype=torch.long)
        self.business_indices = torch.tensor(df['business_idx'].values, dtype=torch.long)
        self.review_texts = df['text'].values.tolist()
        self.stars = torch.tensor(df['stars'].values, dtype=torch.float)
    def __len__(self):
        return len(self.stars)
    def __getitem__(self, idx):
        return self.user_indices[idx], self.business_indices[idx], self.review_texts[idx], self.stars[idx]
print("데이터 로드 및 전처리 완료.")

# 2단계: PyABSA ATE 모델 로드 및 동적 어휘집 구축
print("\n--- 2단계: PyABSA ATE 모델 로드 및 동적 어휘집 구축 시작 ---")

ate_extractor = None
try:
    # Quickstart 예시를 사용하여 ATEPC 모델 로드
    # 'multilingual' 체크포인트를 사용하며, auto_device=True로 GPU/CPU 자동 감지
    ate_extractor = ATEPC.AspectExtractor('multilingual', auto_device=True) # <-- 이 줄이 Quickstart 기반으로 수정되었습니다.
    print("PyABSA ATEPC 모델 로더를 성공적으로 초기화했습니다.")
except Exception as e:
    print(f"오류: PyABSA ATEPC 모델 로더 초기화 실패: {e}")
    print("PyABSA 설치를 확인하고, 시스템에 Git이 올바르게 설정되어 있는지 다시 확인해주세요.")
    print("모델 로드 실패 시, 속성 추출 모듈은 폴백(Fallback) 로직으로 동작합니다.")
    ate_extractor = None
    
train_review_texts = train_df['text'].values.tolist()
print("\n학습 데이터 리뷰 텍스트에서 속성 키워드 추출 중 (어휘집 구축을 위해)...")
all_extracted_aspect_terms_from_train = []
if ate_extractor:
    results = ate_extractor.predict(
        train_review_texts,
        print_result=False,
        ignore_warnings=True
    )
    for result in results:
        if 'aspect' in result and result['aspect']:
            aspect_list = result['aspect'] if isinstance(result['aspect'], list) else [result['aspect']]
            for term in aspect_list:
                all_extracted_aspect_terms_from_train.append(term.lower())
else:
    print("경고: PyABSA ATE 모델이 로드되지 않아 동적 어휘집 구축이 제한됩니다.")

term_counts = Counter(all_extracted_aspect_terms_from_train)
ASPECT_TO_ID = {'<PAD>': 0, '<UNK>': 1, 'general': 2}
next_id = 3
for term, count in term_counts.most_common():
    if term not in ASPECT_TO_ID:
        ASPECT_TO_ID[term] = next_id
        next_id += 1
if 'general' not in ASPECT_TO_ID:
    ASPECT_TO_ID['general'] = next_id
    next_id += 1
NUM_ASPECT_TERMS = len(ASPECT_TO_ID)
print(f"동적으로 구축된 어휘집 크기 (학습 데이터 기반): {NUM_ASPECT_TERMS}개 키워드")
print(f"어휘집 상위 10개 키워드: {list(ASPECT_TO_ID.items())[3:13]}...")
print("PyABSA 모델 로드 및 동적 어휘집 구축 완료 (학습 데이터 기반).")

# 3단계: AAT-Rec 모델 구성 요소 정의
print("\n--- 3단계: AAT-Rec 모델 구성 요소 정의 시작 ---")

class CustomerRestaurantInteractionModule(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim, mlp_dims):
        super(CustomerRestaurantInteractionModule, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.business_embedding = nn.Embedding(num_businesses, embedding_dim)
        layers = []
        input_dim = embedding_dim * 2
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = input_dim
    def forward(self, user_indices, business_indices):
        user_emb = self.user_embedding(user_indices)
        business_emb = self.business_embedding(business_indices)
        combined_emb = torch.cat((user_emb, business_emb), dim=-1)
        interaction_features = self.mlp(combined_emb)
        return interaction_features

class ReviewAspectTermExtractionModule(nn.Module):
    def __init__(self, num_aspect_terms, aspect_embedding_dim, num_heads, mlp_dims, ate_extractor_instance, aspect_to_id_map):
        super(ReviewAspectTermExtractionModule, self).__init__()
        self.ate_extractor = ate_extractor_instance
        self.aspect_to_id = aspect_to_id_map
        self.aspect_embedding = nn.Embedding(num_aspect_terms, aspect_embedding_dim, padding_idx=0)
        self.multihead_attn = nn.MultiheadAttention(embed_dim=aspect_embedding_dim, num_heads=num_heads, batch_first=True)
        layers = []
        input_dim = aspect_embedding_dim
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = input_dim
    def forward(self, review_texts):
        extracted_aspect_ids_batch = self._extract_aspect_terms_with_pyabsa(review_texts)
        batch_aspect_features = []
        for aspect_ids_for_review in extracted_aspect_ids_batch:
            if not aspect_ids_for_review:
                unk_aspect_id = self.aspect_to_id['<UNK>']
                embedded_terms = self.aspect_embedding(
                    torch.tensor([unk_aspect_id], dtype=torch.long).to(self.aspect_embedding.weight.device)
                )
                attn_output, _ = self.multihead_attn(
                    embedded_terms.unsqueeze(0), embedded_terms.unsqueeze(0), embedded_terms.unsqueeze(0)
                )
                pooled_output = attn_output.squeeze(0)
            else:
                embedded_terms = self.aspect_embedding(
                    torch.tensor(aspect_ids_for_review, dtype=torch.long).to(self.aspect_embedding.weight.device)
                )
                input_tensor = embedded_terms.unsqueeze(0)
                attn_output, _ = self.multihead_attn(input_tensor, input_tensor, input_tensor)
                pooled_output = attn_output.mean(dim=1)
            aspect_features = self.mlp(pooled_output)
            batch_aspect_features.append(aspect_features)
        return torch.cat(batch_aspect_features, dim=0)

    def _extract_aspect_terms_with_pyabsa(self, texts):
        extracted_indices_batch = []
        if self.ate_extractor is None:
            for _ in texts:
                extracted_indices_batch.append([self.aspect_to_id['<UNK>']])
            return extracted_indices_batch
        results = self.ate_extractor.predict(
            texts,
            print_result=False,
            ignore_warnings=True
        )
        for result in results:
            aspect_terms_for_review = []
            if 'aspect' in result and result['aspect']:
                aspect_list = result['aspect'] if isinstance(result['aspect'], list) else [result['aspect']]
                for aspect_term_str in aspect_list:
                    aspect_id = self.aspect_to_id.get(aspect_term_str.lower(), self.aspect_to_id['<UNK>'])
                    aspect_terms_for_review.append(aspect_id)
            if not aspect_terms_for_review:
                aspect_terms_for_review.append(self.aspect_to_id['<UNK>'])
            extracted_indices_batch.append(aspect_terms_for_review)
        return extracted_indices_batch

class RatingPredictionModule(nn.Module):
    def __init__(self, input_dim, mlp_dims):
        super(RatingPredictionModule, self).__init__()
        layers = []
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        layers.append(nn.Linear(input_dim, 1))
        self.mlp = nn.Sequential(*layers)
    def forward(self, combined_features):
        raw_prediction = self.mlp(combined_features)
        prediction = torch.sigmoid(raw_prediction) * 4 + 1
        return prediction

class AATRec(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim,
                 user_biz_mlp_dims, num_aspect_terms, aspect_embedding_dim, num_attn_heads,
                 aspect_mlp_dims, final_mlp_dims, ate_extractor_instance, aspect_to_id_map):
        super(AATRec, self).__init__()
        self.customer_restaurant_module = CustomerRestaurantInteractionModule(
            num_users, num_businesses, embedding_dim, user_biz_mlp_dims
        )
        self.aspect_extraction_module = ReviewAspectTermExtractionModule(
            num_aspect_terms, aspect_embedding_dim, num_attn_heads, aspect_mlp_dims,
            ate_extractor_instance, aspect_to_id_map
        )
        combined_feature_dim = self.customer_restaurant_module.output_dim + \
                               self.aspect_extraction_module.output_dim
        self.rating_prediction_module = RatingPredictionModule(
            combined_feature_dim, final_mlp_dims
        )
    def forward(self, user_indices, business_indices, review_texts):
        interaction_features = self.customer_restaurant_module(user_indices, business_indices)
        aspect_features = self.aspect_extraction_module(review_texts)
        combined_features = torch.cat((interaction_features, aspect_features), dim=-1)
        predicted_rating = self.rating_prediction_module(combined_features).squeeze()
        return predicted_rating

print("AATRec 모델 구성 요소 정의 완료.")

# 4단계: 모델 학습 및 평가
print("\n--- 4단계: 모델 학습 및 평가 시작 ---")

EMBEDDING_DIM = 64
USER_BIZ_MLP_DIMS = [128, 64]
ASPECT_EMBEDDING_DIM = 128
NUM_ATTN_HEADS = 8
ASPECT_MLP_DIMS = [64, 32]
FINAL_MLP_DIMS = [32, 16]

BATCH_SIZE = 128
train_loader = DataLoader(ReviewDataset(train_df), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(ReviewDataset(val_df), batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(ReviewDataset(test_df), batch_size=BATCH_SIZE, shuffle=False)

model = AATRec(
    num_users, num_businesses, EMBEDDING_DIM,
    USER_BIZ_MLP_DIMS, NUM_ASPECT_TERMS, ASPECT_EMBEDDING_DIM, NUM_ATTN_HEADS,
    ASPECT_MLP_DIMS, FINAL_MLP_DIMS, ate_extractor, ASPECT_TO_ID
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"모델을 '{device}' 장치로 이동했습니다.")

criterion = nn.MSELoss()
LEARNING_RATE = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

NUM_EPOCHS = 50
PATIENCE = 5
MIN_DELTA = 0.001
MODEL_SAVE_PATH = 'best_aatrec_model_pyabsa.pt'

best_val_rmse = float('inf')
epochs_no_improve = 0

print(f"\n모델 학습 시작 (총 {NUM_EPOCHS} 에포크, 배치 크기: {BATCH_SIZE}, 학습률: {LEARNING_RATE})")

for epoch in range(NUM_EPOCHS):
    model.train()
    total_train_loss = 0
    for user_indices, business_indices, review_texts, stars in train_loader:
        user_indices, business_indices, stars = user_indices.to(device), business_indices.to(device), stars.to(device)
        optimizer.zero_grad()
        predicted_stars = model(user_indices, business_indices, review_texts)
        loss = criterion(predicted_stars, stars)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_loader)

    model.eval()
    total_val_loss = 0
    val_preds = []
    val_true = []
    with torch.no_grad():
        for user_indices, business_indices, review_texts, stars in val_loader:
            user_indices, business_indices, stars = user_indices.to(device), business_indices.to(device), stars.to(device)
            predicted_stars = model(user_indices, business_indices, review_texts)
            total_val_loss += criterion(predicted_stars, stars).item()
            val_preds.extend(predicted_stars.cpu().numpy())
            val_true.extend(stars.cpu().numpy())
    avg_val_loss = total_val_loss / len(val_loader)
    current_val_rmse = np.sqrt(mean_squared_error(val_true, val_preds))

    sys.stdout.write(f"\rEpoch {epoch+1}/{NUM_EPOCHS}, "
                     f"Train Loss: {avg_train_loss:.4f}, "
                     f"Val Loss: {avg_val_loss:.4f}, "
                     f"Val RMSE: {current_val_rmse:.4f}")

    if current_val_rmse < best_val_rmse - MIN_DELTA:
        best_val_rmse = current_val_rmse
        epochs_no_improve = 0
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        sys.stdout.write(f" --> 검증 RMSE 개선됨. 최적 모델 저장됨: {best_val_rmse:.4f}\n")
        sys.stdout.flush()
    else:
        epochs_no_improve += 1
        if epochs_no_improve == PATIENCE:
            sys.stdout.write(f" --> 조기 종료! {PATIENCE} 에포크 동안 검증 RMSE 개선이 없었습니다.\n")
            sys.stdout.flush()
            break
        else:
            sys.stdout.write(f" --> 검증 RMSE 개선되지 않음. 대기 중... ({epochs_no_improve}/{PATIENCE})\n")
            sys.stdout.flush()

sys.stdout.write("\n")
print("모델 학습 완료!")

if os.path.exists(MODEL_SAVE_PATH):
    model.load_state_dict(torch.load(MODEL_SAVE_PATH))
    print(f"최적의 모델 가중치 '{MODEL_SAVE_PATH}' 로드 완료.")
else:
    print(f"경고: 최적의 모델 가중치 '{MODEL_SAVE_PATH}'를 찾을 수 없습니다. 현재 모델 상태로 테스트를 진행합니다.")

model.eval()
test_preds = []
test_true = []
with torch.no_grad():
    for user_indices, business_indices, review_texts, stars in test_loader:
        user_indices, business_indices, stars = user_indices.to(device), business_indices.to(device), stars.to(device)
        predicted_stars = model(user_indices, business_indices, review_texts)
        test_preds.extend(predicted_stars.cpu().numpy())
        test_true.extend(stars.cpu().numpy())

mse = mean_squared_error(test_true, test_preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_true, test_preds)
mape = mean_absolute_percentage_error(test_true, test_preds)

print(f"\n--- 최종 모델 성능 평가 (학습률: {LEARNING_RATE}, 배치 크기: {BATCH_SIZE}) ---")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

In [ ]:
import torch
print(torch.__version__)

In [1]:
import pandas as pd
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import sys
from collections import Counter
import os
import time

# PyABSA 라이브러리 임포트 (PyABSA 1.16.25 버전에 맞춤)
from pyabsa.functional import ATEPCModelList, ATEPCTrainer
from pyabsa.functional import ATEPCConfigManager
from pyabsa.functional import ABSADatasetList

# MAPE를 위한 유틸리티 함수 (0으로 나누는 오류 방지)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_true = y_true != 0
    if np.sum(non_zero_true) == 0:
        return np.nan # 모든 y_true가 0인 경우 NaN 반환
    return np.mean(np.abs((y_true[non_zero_true] - y_pred[non_zero_true]) / y_true[non_zero_true])) * 100

# 1단계: 데이터 로드 및 전처리
print("--- 1단계: 데이터 로드 및 전처리 시작 ---")
try:
    data = []
    with open('review.json', 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    print(f"'{len(df)}'개의 리뷰 데이터를 성공적으로 로드했습니다.")
except FileNotFoundError:
    print("오류: 'review.json' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    sys.exit()

# --- 데이터셋 크기 조절 ---
SUBSET_SIZE = 50 # 테스트를 위한 데이터의 총 개수를 50개로 더욱 축소
if len(df) > SUBSET_SIZE:
    df = df.sample(n=SUBSET_SIZE, random_state=42).reset_index(drop=True)
    print(f"DEBUG: 데이터셋 크기를 {SUBSET_SIZE}개 리뷰로 축소했습니다.")

user_encoder = LabelEncoder()
business_encoder = LabelEncoder()
df['user_idx'] = user_encoder.fit_transform(df['user_id'])
df['business_idx'] = business_encoder.fit_transform(df['business_id'])
num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)
print(f"'{num_users}'명의 고유 사용자와 '{num_businesses}'개의 고유 레스토랑을 매핑했습니다.")

df['date'] = pd.to_datetime(df['date'], unit='ms')
df = df.sort_values(by='date').reset_index(drop=True)
total_reviews = len(df)
train_split = int(total_reviews * 0.7)
val_split = int(total_reviews * (0.7 + 0.1))
train_df = df.iloc[:train_split].copy()
val_df = df.iloc[train_split:val_split].copy()
test_df = df.iloc[val_split:].copy()
print(f"학습 세트 크기: {len(train_df)}개 리뷰")
print(f"검증 세트 크기: {len(val_df)}개 리뷰")
print(f"테스트 세트 크기: {len(test_df)}개 리뷰")

# 2단계: PyABSA ATE 모델 로드 및 동적 어휘집 구축 (PyABSA 1.16.25 functional 호환)
print("\n--- 2단계: PyABSA ATE 모델 로드 및 동적 어휘집 구축 시작 ---")

ate_extractor = None
try:
    atepc_config = ATEPCConfigManager.get_atepc_config_english()
    atepc_config.model = ATEPCModelList.FAST_LCF_ATEPC
    atepc_config.lcf = 'fusion'
    atepc_config.auto_device = True
    atepc_config.pretrained_bert = 'yangheng/deberta-v3-base-absa-v1.1'
    atepc_config.max_seq_len = 512 # 논문에 명시된 max_seq_len

    ate_extractor = ATEPCTrainer(config=atepc_config,
                                 dataset=ABSADatasetList.Restaurant14, # 내장 데이터셋 중 하나를 더미로 전달
                                 from_checkpoint='',
                                 checkpoint_save_mode=1,
                                 auto_device=True
                                 ).load_trained_model()
    print("PyABSA ATEPC 모델 로더를 성공적으로 초기화했습니다.")
except Exception as e:
    print(f"오류: PyABSA ATEPC 모델 로더 초기화 실패: {e}")
    print("PyABSA 설치를 확인하고, 시스템에 Git이 올바르게 설정되어 있는지 다시 확인해주세요.")
    print("PyABSA 1.16.25 버전이 올바르게 설치되었는지 확인해주세요.")
    print("모델 로드 실패 시, 속성 추출 모듈은 폴백(Fallback) 로직으로 동작합니다.")
    ate_extractor = None

train_review_texts = train_df['text'].values.tolist()
print("\n학습 데이터 리뷰 텍스트에서 속성 키워드 추출 중 (어휘집 구축을 위해)...")
all_extracted_aspect_terms_from_train = []

if ate_extractor:
    initial_results = ate_extractor.extract_aspect(
        inference_source=train_review_texts,
        print_result=False,
    )

    if initial_results is not None:
        for result in initial_results:
            if 'aspect' in result and result['aspect']:
                aspect_list = result['aspect'] if isinstance(result['aspect'], list) else [result['aspect']]
                for term in aspect_list:
                    all_extracted_aspect_terms_from_train.append(term.lower())
    else:
        print("경고: 초기 어휘집 구축 단계에서 PyABSA.extract_aspect가 None을 반환했습니다. 어휘집 구축이 제한됩니다.")
else:
    print("경고: PyABSA ATE 모델이 로드되지 않아 동적 어휘집 구축이 제한됩니다.")

term_counts = Counter(all_extracted_aspect_terms_from_train)
ASPECT_TO_ID = {'<PAD>': 0, '<UNK>': 1, 'general': 2}
next_id = 3
MIN_TERM_FREQUENCY = 1 # 테스트를 위해 최소 빈도수를 1로 설정하여 어휘집 크기 확보
filtered_terms = [term for term, count in term_counts.items() if count >= MIN_TERM_FREQUENCY]
filtered_terms.sort(key=lambda x: term_counts[x], reverse=True)

for term in filtered_terms:
    if term not in ASPECT_TO_ID:
        ASPECT_TO_ID[term] = next_id
        next_id += 1
if 'general' not in ASPECT_TO_ID:
    ASPECT_TO_ID['general'] = next_id
    next_id += 1
NUM_ASPECT_TERMS = len(ASPECT_TO_ID)
print(f"동적으로 구축된 어휘집 크기 (학습 데이터 기반): {NUM_ASPECT_TERMS}개 키워드")
print(f"어휘집 상위 10개 키워드: {list(ASPECT_TO_ID.items())[3:13]}...")
print("PyABSA 모델 로드 및 동적 어휘집 구축 완료 (학습 데이터 기반).")

# --- 2.5단계: 모든 리뷰 텍스트에 대해 PyABSA 속성 사전 추출 및 캐싱 ---
print("\n--- 2.5단계: PyABSA 속성 사전 추출 및 캐싱 시작 ---")

# 이 함수는 ReviewAspectTermExtractionModule의 _extract_aspect_terms_with_pyabsa와 유사하지만
# Dataset.__init__ 단계에서 전체 데이터를 미리 처리하기 위해 별도로 정의
def pre_extract_aspect_ids_for_df(df_subset, ate_extractor_instance, aspect_to_id_map, device_for_pyabsa):
    all_texts = df_subset['text'].values.tolist()
    extracted_indices_batch = []
    if ate_extractor_instance is None:
        for _ in all_texts:
            extracted_indices_batch.append([aspect_to_id_map['<UNK>']])
        return extracted_indices_batch
            
    SUB_BATCH_SIZE_PYABSA = 32 # PyABSA의 내부 처리 배치 크기, GPU 메모리에 따라 조절
    
    # tqdm을 사용하여 진행률 표시 (설치 필요: pip install tqdm)
    # from tqdm import tqdm
    # for i in tqdm(range(0, len(all_texts), SUB_BATCH_SIZE_PYABSA), desc="PyABSA Pre-extraction"):

    for i in range(0, len(all_texts), SUB_BATCH_SIZE_PYABSA):
        sub_texts = all_texts[i:i + SUB_BATCH_SIZE_PYABSA]
        
        sub_results = None
        retry_count = 0
        MAX_RETRIES = 3

        while retry_count < MAX_RETRIES:
            try:
                # PyABSA 1.16.25의 extract_aspect는 device 인자를 직접 받지 않음
                # config.auto_device=True가 이를 처리함
                sub_results = ate_extractor_instance.extract_aspect(
                    inference_source=sub_texts,
                    print_result=False,
                )
                if sub_results is not None:
                    break
                else:
                    print(f"\n경고: PyABSA.extract_aspect가 None을 반환했습니다. 재시도 ({retry_count+1}/{MAX_RETRIES})...")
                    retry_count += 1
                    torch.cuda.empty_cache()
                    time.sleep(1)
            except Exception as e:
                print(f"\n경고: PyABSA.extract_aspect 중 예외 발생: {e}. 재시도 ({retry_count+1}/{MAX_RETRIES})...")
                sub_results = None
                retry_count += 1
                torch.cuda.empty_cache()
                time.sleep(1)
        
        if sub_results is None:
            print(f"\n오류: PyABSA.extract_aspect가 {MAX_RETRIES}번 재시도 후에도 실패했습니다. 해당 서브 배치에 대해 <UNK> 처리합니다.")
            for _ in sub_texts:
                extracted_indices_batch.append([aspect_to_id_map['<UNK>']])
            continue

        for result in sub_results:
            aspect_terms_for_review = []
            if 'aspect' in result and result['aspect']:
                aspect_list = result['aspect'] if isinstance(result['aspect'], list) else [result['aspect']]
                for aspect_term_str in aspect_list:
                    aspect_id = aspect_to_id_map.get(aspect_term_str.lower(), aspect_to_id_map['<UNK>'])
                    aspect_terms_for_review.append(aspect_id)
            if not aspect_terms_for_review:
                aspect_terms_for_review.append(aspect_to_id_map['<UNK>'])
            extracted_indices_batch.append(aspect_terms_for_review)
    return extracted_indices_batch

# PyABSA 추출 시 사용할 디바이스 설정
pyabsa_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_df['extracted_aspect_ids'] = pre_extract_aspect_ids_for_df(train_df, ate_extractor, ASPECT_TO_ID, pyabsa_device)
val_df['extracted_aspect_ids'] = pre_extract_aspect_ids_for_df(val_df, ate_extractor, ASPECT_TO_ID, pyabsa_device)
test_df['extracted_aspect_ids'] = pre_extract_aspect_ids_for_df(test_df, ate_extractor, ASPECT_TO_ID, pyabsa_device)

print("PyABSA 속성 사전 추출 및 캐싱 완료.")

class ReviewDataset(Dataset):
    def __init__(self, df):
        self.user_indices = torch.tensor(df['user_idx'].values, dtype=torch.long)
        self.business_indices = torch.tensor(df['business_idx'].values, dtype=torch.long)
        # 원본 review_texts는 이제 필요 없음
        self.extracted_aspect_ids = df['extracted_aspect_ids'].values.tolist() # 미리 추출된 ID 사용
        self.stars = torch.tensor(df['stars'].values, dtype=torch.float)
    def __len__(self):
        return len(self.stars)
    def __getitem__(self, idx):
        # review_texts 대신 추출된 aspect_ids를 반환
        return self.user_indices[idx], self.business_indices[idx], self.extracted_aspect_ids[idx], self.stars[idx]

# 3단계: AAT-Rec 모델 구성 요소 정의
print("\n--- 3단계: AAT-Rec 모델 구성 요소 정의 시작 ---")

class CustomerRestaurantInteractionModule(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim, mlp_dims):
        super(CustomerRestaurantInteractionModule, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.business_embedding = nn.Embedding(num_businesses, embedding_dim)
        layers = []
        input_dim = embedding_dim * 2
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = input_dim
    def forward(self, user_indices, business_indices):
        user_emb = self.user_embedding(user_indices)
        business_emb = self.business_embedding(business_indices)
        combined_emb = torch.cat((user_emb, business_emb), dim=-1)
        interaction_features = self.mlp(combined_emb)
        return interaction_features

class ReviewAspectTermExtractionModule(nn.Module):
    def __init__(self, num_aspect_terms, aspect_embedding_dim, num_heads, mlp_dims):
        super(ReviewAspectTermExtractionModule, self).__init__()
        # ate_extractor_instance와 aspect_to_id_map는 더 이상 이 모듈의 __init__에서 받지 않음
        # (미리 추출된 ID를 사용하므로 PyABSA 모델이 더 이상 필요 없음)
        self.aspect_embedding = nn.Embedding(num_aspect_terms, aspect_embedding_dim, padding_idx=0)
        self.multihead_attn = nn.MultiheadAttention(embed_dim=aspect_embedding_dim, num_heads=num_heads, batch_first=True)
        layers = []
        input_dim = aspect_embedding_dim
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = input_dim
    
    # review_texts 대신 미리 추출된 aspect_ids 리스트를 직접 입력으로 받음
    def forward(self, pre_extracted_aspect_ids_batch):
        batch_aspect_features = []
        # 각 리뷰의 속성 ID 리스트에 대해 처리
        for aspect_ids_for_review in pre_extracted_aspect_ids_batch:
            # PyTorch 텐서로 변환
            # ReviewDataset의 collate_fn이 없으므로, DataLoader는 이 리스트를 그대로 전달
            # 따라서 각 요소는 길이가 다른 리스트가 될 수 있음
            # MultiheadAttention은 (Batch, SeqLen, EmbDim) 형태를 기대하므로,
            # 여기서는 각 리뷰의 속성들을 개별적으로 처리하여 unsqueeze(0)로 배치 차원 추가
            
            # 비어있는 경우 (추출된 속성이 없는 경우) UNK 토큰 사용
            if not aspect_ids_for_review:
                # 미리 pre_extract_aspect_ids_for_df에서 <UNK>를 넣어주므로 이 분기는 거의 타지 않음.
                # 하지만 안전을 위해 UNK 처리 로직 유지 (ID 1이 <UNK> 가정)
                embedded_terms = self.aspect_embedding(
                    torch.tensor([1], dtype=torch.long).to(self.aspect_embedding.weight.device)
                )
            else:
                embedded_terms = self.aspect_embedding(
                    torch.tensor(aspect_ids_for_review, dtype=torch.long).to(self.aspect_embedding.weight.device)
                )
            
            input_tensor = embedded_terms.unsqueeze(0) # (1, num_aspect_terms, embedding_dim)
            attn_output, _ = self.multihead_attn(input_tensor, input_tensor, input_tensor)
            pooled_output = attn_output.mean(dim=1) # (1, embedding_dim)
            aspect_features = self.mlp(pooled_output) # (1, output_dim)
            batch_aspect_features.append(aspect_features)
        
        # 모든 리뷰의 속성 특징을 하나의 배치 텐서로 결합
        return torch.cat(batch_aspect_features, dim=0)

    # 이 메서드는 이제 ReviewDataset 초기화 단계에서만 사용되므로, 
    # ReviewAspectTermExtractionModule 클래스에서 제거하거나 비활성화해도 됨.
    # 코드의 가독성을 위해 일단 주석 처리하여 '더 이상 사용되지 않음'을 명시
    # def _extract_aspect_terms_with_pyabsa(...):
    #     pass


class RatingPredictionModule(nn.Module):
    def __init__(self, input_dim, mlp_dims):
        super(RatingPredictionModule, self).__init__()
        layers = []
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        layers.append(nn.Linear(input_dim, 1))
        self.mlp = nn.Sequential(*layers)
    def forward(self, combined_features):
        raw_prediction = self.mlp(combined_features)
        prediction = torch.sigmoid(raw_prediction) * 4 + 1 # 1~5점 스케일로 정규화
        return prediction

class AATRec(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim,
                 user_biz_mlp_dims, num_aspect_terms, aspect_embedding_dim, num_attn_heads,
                 aspect_mlp_dims, final_mlp_dims): # ate_extractor_instance, aspect_to_id_map 인자 제거
        super(AATRec, self).__init__()
        self.customer_restaurant_module = CustomerRestaurantInteractionModule(
            num_users, num_businesses, embedding_dim, user_biz_mlp_dims
        )
        # ate_extractor_instance, aspect_to_id_map 인자 제거
        self.aspect_extraction_module = ReviewAspectTermExtractionModule(
            num_aspect_terms, aspect_embedding_dim, num_attn_heads, aspect_mlp_dims
        )
        combined_feature_dim = self.customer_restaurant_module.output_dim + \
                               self.aspect_extraction_module.output_dim
        self.rating_prediction_module = RatingPredictionModule(
            combined_feature_dim, final_mlp_dims
        )
    # review_texts 대신 pre_extracted_aspect_ids를 입력으로 받음
    def forward(self, user_indices, business_indices, pre_extracted_aspect_ids):
        interaction_features = self.customer_restaurant_module(user_indices, business_indices)
        aspect_features = self.aspect_extraction_module(pre_extracted_aspect_ids)
        combined_features = torch.cat((interaction_features, aspect_features), dim=-1)
        predicted_rating = self.rating_prediction_module(combined_features).squeeze()
        return predicted_rating

print("AATRec 모델 구성 요소 정의 완료.")

# 4단계: 모델 학습 및 평가
print("\n--- 4단계: 모델 학습 및 평가 시작 ---")

EMBEDDING_DIM = 64
USER_BIZ_MLP_DIMS = [128, 64]
ASPECT_EMBEDDING_DIM = 128
NUM_ATTN_HEADS = 8
ASPECT_MLP_DIMS = [64, 32]
FINAL_MLP_DIMS = [32, 16]

# DataLoader의 batch_size는 ReviewDataset에서 반환하는 데이터 구조에 따라 유연하게 설정
# 현재 aspect_ids가 가변 길이 리스트이므로, collate_fn을 정의하지 않으면 batch_size=1이 강제될 수 있음
# 작은 데이터셋이므로 일단 기본 동작을 유지하며, 필요시 collate_fn 구현 고려
BATCH_SIZE = 1 # 가변 길이 시퀀스 때문에 기본적으로 1로 설정하는 것이 안전.
                # 그러나 PyTorch 2.x 이상에서는 일부 collate_fn이 기본적으로 작동하기도 함.
                # 에러 발생 시 BATCH_SIZE=1로 변경하거나 collate_fn 구현 필요
train_loader = DataLoader(ReviewDataset(train_df), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(ReviewDataset(val_df), batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(ReviewDataset(test_df), batch_size=BATCH_SIZE, shuffle=False)

model = AATRec(
    num_users, num_businesses, EMBEDDING_DIM,
    USER_BIZ_MLP_DIMS, NUM_ASPECT_TERMS, ASPECT_EMBEDDING_DIM, NUM_ATTN_HEADS,
    ASPECT_MLP_DIMS, FINAL_MLP_DIMS
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"모델을 '{device}' 장치로 이동했습니다.")

criterion = nn.MSELoss()
LEARNING_RATE = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

NUM_EPOCHS = 50
PATIENCE = 5
MIN_DELTA = 0.001
MODEL_SAVE_PATH = 'best_aatrec_model_pyabsa.pt'

best_val_rmse = float('inf')
epochs_no_improve = 0

print(f"\n모델 학습 시작 (총 {NUM_EPOCHS} 에포크, 배치 크기: {BATCH_SIZE}, 학습률: {LEARNING_RATE})")

for epoch in range(NUM_EPOCHS):
    model.train()
    total_train_loss = 0
    for user_indices, business_indices, extracted_aspect_ids, stars in train_loader:
        user_indices, business_indices, stars = user_indices.to(device), business_indices.to(device), stars.to(device)
        # extracted_aspect_ids는 텐서가 아닌 파이썬 리스트의 리스트 형태이므로 to(device) 적용 불가
        # 이 부분은 ReviewAspectTermExtractionModule 내부에서 처리됨 (개별 텐서로 변환 후 to(device))
        
        optimizer.zero_grad()
        predicted_stars = model(user_indices, business_indices, extracted_aspect_ids)
        loss = criterion(predicted_stars, stars)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_loader)

    model.eval()
    total_val_loss = 0
    val_preds = []
    val_true = []
    with torch.no_grad():
        for user_indices, business_indices, extracted_aspect_ids, stars in val_loader:
            user_indices, business_indices, stars = user_indices.to(device), business_indices.to(device), stars.to(device)
            predicted_stars = model(user_indices, business_indices, extracted_aspect_ids)
            total_val_loss += criterion(predicted_stars, stars).item()
            val_preds.extend(predicted_stars.cpu().numpy())
            val_true.extend(stars.cpu().numpy())
    avg_val_loss = total_val_loss / len(val_loader)
    current_val_rmse = np.sqrt(mean_squared_error(val_true, val_preds))

    sys.stdout.write(f"\rEpoch {epoch+1}/{NUM_EPOCHS}, "
                     f"Train Loss: {avg_train_loss:.4f}, "
                     f"Val Loss: {avg_val_loss:.4f}, "
                     f"Val RMSE: {current_val_rmse:.4f}")

    if current_val_rmse < best_val_rmse - MIN_DELTA:
        best_val_rmse = current_val_rmse
        epochs_no_improve = 0
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        sys.stdout.write(f" --> 검증 RMSE 개선됨. 최적 모델 저장됨: {best_val_rmse:.4f}\n")
        sys.stdout.flush()
    else:
        epochs_no_improve += 1
        if epochs_no_improve == PATIENCE:
            sys.stdout.write(f" --> 조기 종료! {PATIENCE} 에포크 동안 검증 RMSE 개선이 없었습니다.\n")
            sys.stdout.flush()
            break
        else:
            sys.stdout.write(f" --> 검증 RMSE 개선되지 않음. 대기 중... ({epochs_no_improve}/{PATIENCE})\n")
            sys.stdout.flush()

sys.stdout.write("\n")
print("모델 학습 완료!")

if os.path.exists(MODEL_SAVE_PATH):
    model.load_state_dict(torch.load(MODEL_SAVE_PATH))
    print(f"최적의 모델 가중치 '{MODEL_SAVE_PATH}' 로드 완료.")
else:
    print(f"경고: 최적의 모델 가중치 '{MODEL_SAVE_PATH}'를 찾을 수 없습니다. 현재 모델 상태로 테스트를 진행합니다.")

model.eval()
test_preds = []
test_true = []
with torch.no_grad():
    for user_indices, business_indices, extracted_aspect_ids, stars in test_loader:
        user_indices, business_indices, stars = user_indices.to(device), business_indices.to(device), stars.to(device)
        predicted_stars = model(user_indices, business_indices, extracted_aspect_ids)
        test_preds.extend(predicted_stars.cpu().numpy())
        test_true.extend(stars.cpu().numpy())

mse = mean_squared_error(test_true, test_preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_true, test_preds)
mape = mean_absolute_percentage_error(test_true, test_preds)

print(f"\n--- 최종 모델 성능 평가 (학습률: {LEARNING_RATE}, 배치 크기: {BATCH_SIZE}) ---")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

C:\Users\alche\anaconda3\envs\my_absa_project_py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Version 1.16.25 of pyabsa is outdated. Version 2.4.1 was released Friday February 23, 2024.
check release notes at https://github.com/yangheng95/PyABSA/blob/release/release-note.json
--- 1단계: 데이터 로드 및 전처리 시작 ---
'447796'개의 리뷰 데이터를 성공적으로 로드했습니다.
DEBUG: 데이터셋 크기를 50개 리뷰로 축소했습니다.
'50'명의 고유 사용자와 '50'개의 고유 레스토랑을 매핑했습니다.
학습 세트 크기: 35개 리뷰
검증 세트 크기: 5개 리뷰
테스트 세트 크기: 10개 리뷰

--- 2단계: PyABSA ATE 모델 로드 및 동적 어휘집 구축 시작 ---


C:\Users\alche\anaconda3\envs\my_absa_project_py310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Remote ABSADataset version: 2022.10.25 Local ABSADatasets version: 2022.10.25
Dataset is not a path, treat dataset as keywords to Load 114.Restaurant14 from: 114.Restaurant14 or Search https://github.com/yangheng95/ABSADatasets locally using findfile
You can set load_aug=True in a trainer to augment your dataset (English only yet) and improve performance.
Please use a new folder to perform new text augmentation if the former augmentation exited unexpectedly


Some weights of the model checkpoint at yangheng/deberta-v3-base-absa-v1.1 were not used when initializing DebertaV2Model: ['pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'pooler.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading dataset cache: fast_lcf_atepc.Restaurant14.dataset.93573f0f850a88e9b1e40b3b6655765169db8323699cf1d2c823b3ee432591ce.cache
2025-05-26 14:19:38,987 INFO: cuda memory allocated:764963840
2025-05-26 14:19:38,988 INFO: ABSADatasetsVersion:2022.10.25	-->	Calling Count:0
2025-05-26 14:19:38,989 INFO: IOB_label_to_index:{'B-ASP': 1, 'I-ASP': 2, 'O': 3, '[CLS]': 4, '[SEP]': 5}	-->	Calling Count:1
2025-05-26 14:19:38,989 INFO: MV:<metric_visualizer.metric_visualizer.MetricVisualizer object at 0x0000028571556170>	-->	Calling Count:0
2025-05-26 14:19:38,990 INFO: PyABSAVersion:1.16.25	-->	Calling Count:0
2025-05-26 14:19:38,990 INFO: SRD:3	-->	Calling Count:0
2025-05-26 14:19:38,991 INFO: TorchVersion:2.7.0+cu126+cuda12.6	-->	Calling Count:0
2025-05-26 14:19:38,991 INFO: TransformersVersion:4.29.0	-->	Calling Count:0
2025-05-26 14:19:38,992 INFO: auto_device:True	-->	Calling Count:1
2025-05-26 14:19:38,992 INFO: batch_size:16	-->	Calling Count:4
2025-05-26 14:19:38,992 INFO: cache_dataset:

  9%|▉         | 21/226 [07:43<1:15:21, 22.05s/it, Epoch:4 | loss_apc:0.0715 | loss_ate:0.0768 | APC_ACC: 87.92(max:88.46) | APC_F1: 82.81(max:83.07) | ATE_F1: 84.06(max:86.22)]


KeyboardInterrupt: 

In [ ]:
import transformers
import pyabsa
print(pyabsa.__version__)
print(transformers.__version__)

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import sys
from collections import Counter
import os

# MAPE를 위한 유틸리티 함수 (0으로 나누는 오류 방지)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_true = y_true != 0
    if np.sum(non_zero_true) == 0:
        return np.nan # 모든 y_true가 0인 경우 NaN 반환
    return np.mean(np.abs((y_true[non_zero_true] - y_pred[non_zero_true]) / y_true[non_zero_true])) * 100

# --- 0. 파일 경로 설정 ---
review_json_path = 'review.json'
absa_results_path = 'absa_atepc_results.json' # .json 확장자로 변경

# --- 1. 데이터 로드 및 전처리 ---
print("--- 1단계: 데이터 로드 및 전처리 시작 ---")
try:
    data = []
    with open(review_json_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    print(f"'{len(df)}'개의 리뷰 데이터를 성공적으로 로드했습니다.")
except FileNotFoundError:
    print(f"오류: '{review_json_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    sys.exit()
except json.JSONDecodeError as e:
    print(f"오류: '{review_json_path}' 파일 파싱 중 오류 발생: {e}. 파일 내용이 올바른 JSON 형식을 따르는지 확인해주세요.")
    sys.exit()

user_encoder = LabelEncoder()
business_encoder = LabelEncoder()
df['user_idx'] = user_encoder.fit_transform(df['user_id'])
df['business_idx'] = business_encoder.fit_transform(df['business_id'])
num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)
print(f"'{num_users}'명의 고유 사용자와 '{num_businesses}'개의 고유 레스토랑을 매핑했습니다.")

try:
    df['date'] = pd.to_datetime(df['date'], unit='ms') # timestamp in milliseconds
except ValueError:
    df['date'] = pd.to_datetime(df['date']) # assume standard date string format if ms fails

df = df.sort_values(by='date').reset_index(drop=True)
total_reviews = len(df)
train_split = int(total_reviews * 0.7)
val_split_idx = int(total_reviews * (0.7 + 0.1)) # 검증 세트의 끝 인덱스
train_df = df.iloc[:train_split].copy()
val_df = df.iloc[train_split:val_split_idx].copy()
test_df = df.iloc[val_split_idx:].copy() # 나머지 20%를 테스트 세트로
print(f"학습 세트 크기: {len(train_df)}개 리뷰")
print(f"검증 세트 크기: {len(val_df)}개 리뷰")
print(f"테스트 세트 크기: {len(test_df)}개 리뷰")

print("\n--- 2단계: absa_atepc_results.json 파일에서 속성 데이터 로드 및 동적 어휘집 구축 시작 ---")

absa_data = {}
try:
    with open(absa_results_path, 'r', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)
            absa_data[entry['review_id']] = entry['aspects']
    print(f"'{absa_results_path}' 파일에서 {len(absa_data)}개의 리뷰 속성 데이터를 성공적으로 로드했습니다.")
except FileNotFoundError:
    print(f"오류: '{absa_results_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    sys.exit()
except json.JSONDecodeError as e:
    print(f"오류: '{absa_results_path}' 파일 파싱 중 오류 발생: {e}. 파일 내용이 올바른 JSON 형식을 따르는지 확인해주세요.")
    sys.exit()

all_extracted_aspect_terms_from_file = []
all_extracted_sentiments_from_file = []

for review_id in train_df['review_id']:
    aspects = absa_data.get(review_id, [])
    if not aspects: # 속성이 없는 경우 'general' 및 'neutral'로 대체
        all_extracted_aspect_terms_from_file.append('general')
        all_extracted_sentiments_from_file.append('neutral')
    else:
        for aspect in aspects:
            if 'term' in aspect and 'sentiment' in aspect:
                all_extracted_aspect_terms_from_file.append(aspect['term'].lower())
                # 감성 라벨을 소문자로 통일 (Positive, Negative, Neutral)
                all_extracted_sentiments_from_file.append(aspect['sentiment'].lower())

# 속성 키워드 어휘집 구축
term_counts = Counter(all_extracted_aspect_terms_from_file)
# <PAD>는 0번, <UNK>는 1번, 'general'은 2번 인덱스를 가집니다.
ASPECT_TO_ID = {'<PAD>': 0, '<UNK>': 1, 'general': 2}
next_id = 3
MIN_TERM_FREQUENCY = 1 # 어휘집 구축 시 최소 등장 빈도
filtered_terms = [term for term, count in term_counts.items() if count >= MIN_TERM_FREQUENCY]
filtered_terms.sort(key=lambda x: term_counts[x], reverse=True)

for term in filtered_terms:
    if term not in ASPECT_TO_ID:
        ASPECT_TO_ID[term] = next_id
        next_id += 1
NUM_ASPECT_TERMS = len(ASPECT_TO_ID)

# 감성 어휘집 구축 (고정된 긍정, 부정, 중립)
SENTIMENT_TO_ID = {'<PAD>': 0, '<UNK>': 1, 'positive': 2, 'negative': 3, 'neutral': 4}
# PyABSA 결과에 따라 'Positive', 'Negative', 'Neutral'을 소문자로 통일하여 처리
NUM_SENTIMENTS = len(SENTIMENT_TO_ID)

print(f"동적으로 구축된 속성 어휘집 크기 (학습 데이터 기반): {NUM_ASPECT_TERMS}개 키워드")
print(f"어휘집 상위 10개 키워드: {list(ASPECT_TO_ID.items())[3:13]}...") # 0,1,2 제외하고 출력
print(f"구축된 감성 어휘집 크기: {NUM_SENTIMENTS}개 감성 ({list(SENTIMENT_TO_ID.items())})")
print("absa_atepc_results.json 파일에서 속성 데이터 로드 및 동적 어휘집 구축 완료.")

print("\n--- 2.5단계: 로드된 속성 데이터를 DataFrame에 매핑 및 캐싱 시작 ---")

def map_aspect_and_sentiment_ids_from_file(df_subset, absa_data_map, aspect_to_id_map, sentiment_to_id_map):
    extracted_aspect_ids_list = []
    extracted_sentiment_ids_list = []
    
    for _, row in df_subset.iterrows():
        review_id = row['review_id']
        aspects_for_review = absa_data_map.get(review_id, [])
        
        aspect_ids_for_review = []
        sentiment_ids_for_review = []
        
        if aspects_for_review:
            for aspect in aspects_for_review:
                if 'term' in aspect and 'sentiment' in aspect:
                    aspect_id = aspect_to_id_map.get(aspect['term'].lower(), aspect_to_id_map['<UNK>'])
                    sentiment_id = sentiment_to_id_map.get(aspect['sentiment'].lower(), sentiment_to_id_map['<UNK>']) # 감성 ID도 매핑
                    
                    aspect_ids_for_review.append(aspect_id)
                    sentiment_ids_for_review.append(sentiment_id)
        
        if not aspect_ids_for_review: # 추출된 속성이 없으면 'general' 및 'neutral' 감성 사용
            aspect_ids_for_review.append(aspect_to_id_map['general'])
            sentiment_ids_for_review.append(sentiment_to_id_map['neutral']) # 'general'과 함께 'neutral' 감성 할당
            
        extracted_aspect_ids_list.append(aspect_ids_for_review)
        extracted_sentiment_ids_list.append(sentiment_ids_for_review)
        
    return extracted_aspect_ids_list, extracted_sentiment_ids_list

# 각 DataFrame에 속성 ID와 감성 ID 리스트 매핑
train_df['extracted_aspect_ids'], train_df['extracted_sentiment_ids'] = map_aspect_and_sentiment_ids_from_file(train_df, absa_data, ASPECT_TO_ID, SENTIMENT_TO_ID)
val_df['extracted_aspect_ids'], val_df['extracted_sentiment_ids'] = map_aspect_and_sentiment_ids_from_file(val_df, absa_data, ASPECT_TO_ID, SENTIMENT_TO_ID)
test_df['extracted_aspect_ids'], test_df['extracted_sentiment_ids'] = map_aspect_and_sentiment_ids_from_file(test_df, absa_data, ASPECT_TO_ID, SENTIMENT_TO_ID)

print("로드된 속성 데이터를 DataFrame에 매핑 및 캐싱 완료.")

class ReviewDataset(Dataset):
    def __init__(self, df):
        self.user_indices = torch.tensor(df['user_idx'].values, dtype=torch.long)
        self.business_indices = torch.tensor(df['business_idx'].values, dtype=torch.long)
        self.extracted_aspect_ids = df['extracted_aspect_ids'].values.tolist()
        self.extracted_sentiment_ids = df['extracted_sentiment_ids'].values.tolist() # 감성 ID 추가
        self.stars = torch.tensor(df['stars'].values, dtype=torch.float)
    def __len__(self):
        return len(self.stars)
    def __getitem__(self, idx):
        return (self.user_indices[idx], self.business_indices[idx],
                self.extracted_aspect_ids[idx], self.extracted_sentiment_ids[idx], # 감성 ID 반환
                self.stars[idx])

def custom_collate_fn(batch):
    user_indices, business_indices, extracted_aspect_ids_list, extracted_sentiment_ids_list, stars = zip(*batch)

    user_indices = torch.stack(user_indices)
    business_indices = torch.stack(business_indices)
    stars = torch.stack(stars)

    # MultiheadAttention 입력을 위한 패딩 및 마스크 생성
    max_aspect_len = max(len(ids) for ids in extracted_aspect_ids_list)
    
    padded_aspect_ids = [ids + [ASPECT_TO_ID['<PAD>']] * (max_aspect_len - len(ids)) for ids in extracted_aspect_ids_list]
    padded_aspect_ids_tensor = torch.tensor(padded_aspect_ids, dtype=torch.long)
    
    padded_sentiment_ids = [ids + [SENTIMENT_TO_ID['<PAD>']] * (max_aspect_len - len(ids)) for ids in extracted_sentiment_ids_list]
    padded_sentiment_ids_tensor = torch.tensor(padded_sentiment_ids, dtype=torch.long)
    
    # key_padding_mask: True는 무시할 요소 (패딩)
    attn_mask = (padded_aspect_ids_tensor == ASPECT_TO_ID['<PAD>']) # aspect_ids를 기준으로 마스크 생성

    return user_indices, business_indices, padded_aspect_ids_tensor, padded_sentiment_ids_tensor, stars, attn_mask

print("\n--- 3단계: AAT-Rec 모델 구성 요소 정의 (감성 통합) ---")

class CustomerRestaurantInteractionModule(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim, mlp_dims, dropout_rate):
        super(CustomerRestaurantInteractionModule, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.business_embedding = nn.Embedding(num_businesses, embedding_dim)
        layers = []
        input_dim = embedding_dim * 2
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = input_dim

    def forward(self, user_indices, business_indices):
        user_emb = self.user_embedding(user_indices)
        business_emb = self.business_embedding(business_indices)
        combined_emb = torch.cat((user_emb, business_emb), dim=-1)
        interaction_features = self.mlp(combined_emb)
        return interaction_features

# 리뷰 속성별 키워드 추출 모듈 (MultiheadAttention 사용) - 감성 정보 통합
class ReviewAspectTermExtractionModule(nn.Module):
    def __init__(self, num_aspect_terms, aspect_embedding_dim, num_sentiments, sentiment_embedding_dim, num_heads, mlp_dims, dropout_rate):
        super(ReviewAspectTermExtractionModule, self).__init__()
        self.aspect_embedding = nn.Embedding(num_aspect_terms, aspect_embedding_dim, padding_idx=ASPECT_TO_ID['<PAD>'])
        self.sentiment_embedding = nn.Embedding(num_sentiments, sentiment_embedding_dim, padding_idx=SENTIMENT_TO_ID['<PAD>']) # 감성 임베딩 추가
        
        # MultiheadAttention의 embed_dim은 이제 속성 임베딩과 감성 임베딩을 합친 차원이 됩니다.
        combined_embedding_dim = aspect_embedding_dim + sentiment_embedding_dim
        self.multihead_attn = nn.MultiheadAttention(embed_dim=combined_embedding_dim, num_heads=num_heads, batch_first=True)
        
        layers = []
        input_dim = combined_embedding_dim # MultiheadAttention 출력 차원은 combined_embedding_dim과 동일
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = input_dim
            
    def forward(self, pre_extracted_aspect_ids_batch, pre_extracted_sentiment_ids_batch, attn_mask):
        embedded_terms = self.aspect_embedding(pre_extracted_aspect_ids_batch)
        embedded_sentiments = self.sentiment_embedding(pre_extracted_sentiment_ids_batch)
        
        # 속성 임베딩과 감성 임베딩을 결합 (Concatenate)
        combined_embeddings = torch.cat((embedded_terms, embedded_sentiments), dim=-1) # dim=-1은 마지막 차원 기준으로 합침
        
        # MultiheadAttention에 key_padding_mask 적용
        attn_output, _ = self.multihead_attn(
            combined_embeddings, 
            combined_embeddings, 
            combined_embeddings, 
            key_padding_mask=attn_mask
        )

        # 패딩된 요소들을 0으로 만들고 유효한 요소들만 평균 풀링
        valid_attn_output = attn_output * (~attn_mask.unsqueeze(-1)).float()
        pooled_output = valid_attn_output.sum(dim=1) / (~attn_mask).sum(dim=1, keepdim=True).float().clamp(min=1)
        
        aspect_features = self.mlp(pooled_output)
        return aspect_features

class RatingPredictionModule(nn.Module):
    def __init__(self, input_dim, mlp_dims, dropout_rate):
        super(RatingPredictionModule, self).__init__()
        layers = []
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))
            input_dim = dim
        layers.append(nn.Linear(input_dim, 1))
        self.mlp = nn.Sequential(*layers)
    def forward(self, combined_features):
        raw_prediction = self.mlp(combined_features)
        prediction = torch.sigmoid(raw_prediction) * 4 + 1 # 1~5점 스케일로 정규화
        return prediction

class AATRec(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim,
                 user_biz_mlp_dims, num_aspect_terms, aspect_embedding_dim,
                 num_sentiments, sentiment_embedding_dim, # 감성 관련 파라미터 추가
                 num_attn_heads, aspect_mlp_dims, final_mlp_dims, dropout_rate):
        super(AATRec, self).__init__()
        self.customer_restaurant_module = CustomerRestaurantInteractionModule(
            num_users, num_businesses, embedding_dim, user_biz_mlp_dims, dropout_rate
        )
        self.aspect_extraction_module = ReviewAspectTermExtractionModule(
            num_aspect_terms, aspect_embedding_dim, num_sentiments, sentiment_embedding_dim, # 감성 관련 파라미터 전달
            num_attn_heads, aspect_mlp_dims, dropout_rate
        )
        combined_feature_dim = self.customer_restaurant_module.output_dim + \
                               self.aspect_extraction_module.output_dim
        self.rating_prediction_module = RatingPredictionModule(
            combined_feature_dim, final_mlp_dims, dropout_rate
        )
    def forward(self, user_indices, business_indices, pre_extracted_aspect_ids, pre_extracted_sentiment_ids, attn_mask): # 감성 ID 추가
        interaction_features = self.customer_restaurant_module(user_indices, business_indices)
        aspect_features = self.aspect_extraction_module(pre_extracted_aspect_ids, pre_extracted_sentiment_ids, attn_mask) # 감성 ID 전달
        combined_features = torch.cat((interaction_features, aspect_features), dim=-1)
        predicted_rating = self.rating_prediction_module(combined_features).squeeze()
        return predicted_rating

print("AATRec 모델 구성 요소 정의 완료.")


# --- 4. 하이퍼파라미터 설정 및 모델 학습 ---

print("\n--- 4단계: 단일 하이퍼파라미터 조합으로 모델 학습 및 저장 시작 ---")

# Dataset 인스턴스 생성
train_dataset = ReviewDataset(train_df)
val_dataset = ReviewDataset(val_df)
test_dataset = ReviewDataset(test_df) # 테스트는 학습 후 최종 평가에만 사용

# 학습에 사용할 단일 하이퍼파라미터 조합 설정
best_params = {
    'embedding_dim': 64,
    'aspect_embedding_dim': 96, # 속성 임베딩 차원
    'sentiment_embedding_dim': 32, # 감성 임베딩 차원 (새로 추가)
    'num_attn_heads': 8,
    'learning_rate': 0.001,
    'batch_size': 128,
    'dropout_rate': 0.2,
    'user_biz_mlp_dims': [128, 64],
    'aspect_mlp_dims': [64, 32],
    'final_mlp_dims': [32, 16]
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"학습 장치: '{device}'")

best_overall_rmse = float('inf') # 최적 모델 저장을 위한 초기화
best_model_path = 'final_best_aatrec_model_multihead_aspect_sentiment.pt' # 최종 최적 모델 저장 경로 (이름 변경)

print(f"현재 파라미터: {best_params}")

# 현재 조합의 하이퍼파라미터 설정
EMBEDDING_DIM = best_params['embedding_dim']
ASPECT_EMBEDDING_DIM = best_params['aspect_embedding_dim']
SENTIMENT_EMBEDDING_DIM = best_params['sentiment_embedding_dim'] # 새로운 파라미터
NUM_ATTN_HEADS = best_params['num_attn_heads']
LEARNING_RATE = best_params['learning_rate']
BATCH_SIZE = best_params['batch_size']
DROPOUT_RATE = best_params['dropout_rate']
USER_BIZ_MLP_DIMS = best_params['user_biz_mlp_dims']
ASPECT_MLP_DIMS = best_params['aspect_mlp_dims']
FINAL_MLP_DIMS = best_params['final_mlp_dims']

# DataLoader 생성 (custom_collate_fn 다시 사용)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate_fn)

model = AATRec(
    num_users, num_businesses, EMBEDDING_DIM,
    USER_BIZ_MLP_DIMS, NUM_ASPECT_TERMS, ASPECT_EMBEDDING_DIM,
    NUM_SENTIMENTS, SENTIMENT_EMBEDDING_DIM, # 감성 관련 파라미터 전달
    NUM_ATTN_HEADS, ASPECT_MLP_DIMS, FINAL_MLP_DIMS, DROPOUT_RATE
)
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 조기 종료 설정
PATIENCE = 5 # 조기 종료 대기 횟수
MIN_DELTA = 0.001 # 검증 RMSE 개선을 위한 최소 변화량
current_best_val_rmse = float('inf') # 'current_best_val_rmse' 초기화
epochs_no_improve = 0

NUM_EPOCHS_PER_COMBINATION = 50 # 최대 에포크

print("\n--- 모델 학습 시작 ---")
for epoch in range(NUM_EPOCHS_PER_COMBINATION):
    model.train()
    total_train_loss = 0
    for user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars, attn_mask in train_loader: # 감성 ID 추가
        user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars = \
            user_indices.to(device), business_indices.to(device), extracted_aspect_ids.to(device), extracted_sentiment_ids.to(device), stars.to(device) # 감성 ID to device
        attn_mask = attn_mask.to(device)
        
        optimizer.zero_grad()
        predicted_stars = model(user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, attn_mask) # 감성 ID 전달
        loss = criterion(predicted_stars, stars)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_loader)

    model.eval()
    total_val_loss = 0
    val_preds = []
    val_true = []
    with torch.no_grad():
        for user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars, attn_mask in val_loader: # 감성 ID 추가
            user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars = \
                user_indices.to(device), business_indices.to(device), extracted_aspect_ids.to(device), extracted_sentiment_ids.to(device), stars.to(device) # 감성 ID to device
            attn_mask = attn_mask.to(device)
            predicted_stars = model(user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, attn_mask) # 감성 ID 전달
            total_val_loss += criterion(predicted_stars, stars).item()
            val_preds.extend(predicted_stars.cpu().numpy())
            val_true.extend(stars.cpu().numpy())
    avg_val_loss = total_val_loss / len(val_loader)
    
    # 모델 출력에서 1~5점 스케일링이 이미 되어 있으므로, 추가 클리핑은 선택 사항이지만 안전을 위해 유지
    val_preds_clipped = np.clip(val_preds, 1.0, 5.0)
    current_val_rmse = np.sqrt(mean_squared_error(val_true, val_preds_clipped))

    sys.stdout.write(f"\r  Epoch {epoch+1}/{NUM_EPOCHS_PER_COMBINATION}, "
                     f"Train Loss: {avg_train_loss:.4f}, "
                     f"Val Loss: {avg_val_loss:.4f}, "
                     f"Val RMSE: {current_val_rmse:.4f}")
    sys.stdout.flush()

    if current_val_rmse < current_best_val_rmse - MIN_DELTA:
        current_best_val_rmse = current_val_rmse
        epochs_no_improve = 0
        torch.save(model.state_dict(), best_model_path) # 최적 모델 저장
        sys.stdout.write(f" --> 최적 검증 RMSE 개선됨: {current_best_val_rmse:.4f}. 모델 저장됨.\n")
        sys.stdout.flush()
    else:
        epochs_no_improve += 1
        if epochs_no_improve == PATIENCE:
            sys.stdout.write(f" --> 조기 종료! {PATIENCE} 에포크 동안 검증 RMSE 개선이 없었습니다.\n")
            sys.stdout.flush()
            break
            
print("\n--- 모델 학습 완료 ---")
print(f"최종 저장된 모델의 검증 RMSE: {current_best_val_rmse:.4f}")
print(f"저장 경로: {best_model_path}")


# --- 최종 모델 테스트 ---
print("\n--- 최종 모델 테스트 시작 (최적 파라미터 사용) ---")

if os.path.exists(best_model_path) and best_params:
    # 최적 파라미터로 모델 재구성 (저장된 모델 로드)
    final_model = AATRec(
        num_users, num_businesses, best_params['embedding_dim'],
        best_params['user_biz_mlp_dims'], NUM_ASPECT_TERMS, best_params['aspect_embedding_dim'],
        NUM_SENTIMENTS, best_params['sentiment_embedding_dim'], # 감성 관련 파라미터 전달
        best_params['num_attn_heads'], best_params['aspect_mlp_dims'], best_params['final_mlp_dims'], best_params['dropout_rate']
    )
    final_model.load_state_dict(torch.load(best_model_path, map_location=device)) # map_location 추가
    final_model.to(device)
    print(f"최적의 모델 가중치 '{best_model_path}' 로드 완료.")
else:
    print("경고: 최적의 모델 가중치를 찾을 수 없거나 최적 파라미터가 설정되지 않았습니다. 테스트를 건너뜜.")
    sys.exit()

# 테스트 로더도 custom_collate_fn 사용
test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'], shuffle=False, collate_fn=custom_collate_fn)

final_model.eval()
test_preds = []
test_true = []
with torch.no_grad():
    for user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars, attn_mask in test_loader: # 감성 ID 추가
        user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars = \
            user_indices.to(device), business_indices.to(device), extracted_aspect_ids.to(device), extracted_sentiment_ids.to(device), stars.to(device) # 감성 ID to device
        attn_mask = attn_mask.to(device)
        predicted_stars = final_model(user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, attn_mask) # 감성 ID 전달
        test_preds.extend(predicted_stars.cpu().numpy())
        test_true.extend(stars.cpu().numpy())

# 예측 범위 클리핑 (1점 ~ 5점)
test_preds_clipped = np.clip(test_preds, 1.0, 5.0)

mse = mean_squared_error(test_true, test_preds_clipped)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_true, test_preds_clipped)
mape = mean_absolute_percentage_error(test_true, test_preds_clipped)

print(f"\n--- 최종 모델 성능 평가 (최적 파라미터) ---")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"사용된 최적 하이퍼파라미터: {best_params}")

--- 1단계: 데이터 로드 및 전처리 시작 ---
'447796'개의 리뷰 데이터를 성공적으로 로드했습니다.
'27807'명의 고유 사용자와 '6831'개의 고유 레스토랑을 매핑했습니다.
학습 세트 크기: 313457개 리뷰
검증 세트 크기: 44779개 리뷰
테스트 세트 크기: 89560개 리뷰

--- 2단계: absa_atepc_results.json 파일에서 속성 데이터 로드 및 동적 어휘집 구축 시작 ---
'absa_atepc_results.json' 파일에서 447796개의 리뷰 속성 데이터를 성공적으로 로드했습니다.
동적으로 구축된 속성 어휘집 크기 (학습 데이터 기반): 39160개 키워드
어휘집 상위 10개 키워드: [('food', 3), ('service', 4), ('staff', 5), ('place', 6), ('atmosphere', 7), ('prices', 8), ('they', 9), ('price', 10), ('pizza', 11), ('it', 12)]...
구축된 감성 어휘집 크기: 5개 감성 ([('<PAD>', 0), ('<UNK>', 1), ('positive', 2), ('negative', 3), ('neutral', 4)])
absa_atepc_results.json 파일에서 속성 데이터 로드 및 동적 어휘집 구축 완료.

--- 2.5단계: 로드된 속성 데이터를 DataFrame에 매핑 및 캐싱 시작 ---
로드된 속성 데이터를 DataFrame에 매핑 및 캐싱 완료.

--- 3단계: AAT-Rec 모델 구성 요소 정의 (감성 통합) ---
AATRec 모델 구성 요소 정의 완료.

--- 4단계: 단일 하이퍼파라미터 조합으로 모델 학습 및 저장 시작 ---
학습 장치: 'cuda'
현재 파라미터: {'embedding_dim': 64, 'aspect_embedding_dim': 96, 'sentiment_embedding_dim': 32, 'num_attn_heads': 8, 'learning_rate

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import sys
from collections import Counter
import os
import itertools # 조합 생성을 위해 추가

# MAPE를 위한 유틸리티 함수 (0으로 나누는 오류 방지)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_true = y_true != 0
    if np.sum(non_zero_true) == 0:
        return np.nan # 모든 y_true가 0인 경우 NaN 반환
    return np.mean(np.abs((y_true[non_zero_true] - y_pred[non_zero_true]) / y_true[non_zero_true])) * 100

# --- 0. 파일 경로 설정 ---
review_json_path = 'review.json'
absa_results_path = 'absa_atepc_results.json' 

# --- 1. 데이터 로드 및 전처리 ---
print("--- 1단계: 데이터 로드 및 전처리 시작 ---")
try:
    data = []
    with open(review_json_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    print(f"'{len(df)}'개의 리뷰 데이터를 성공적으로 로드했습니다.")
except FileNotFoundError:
    print(f"오류: '{review_json_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    sys.exit()
except json.JSONDecodeError as e:
    print(f"오류: '{review_json_path}' 파일 파싱 중 오류 발생: {e}. 파일 내용이 올바른 JSON 형식을 따르는지 확인해주세요.")
    sys.exit()

user_encoder = LabelEncoder()
business_encoder = LabelEncoder()
df['user_idx'] = user_encoder.fit_transform(df['user_id'])
df['business_idx'] = business_encoder.fit_transform(df['business_id'])
num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)
print(f"'{num_users}'명의 고유 사용자와 '{num_businesses}'개의 고유 레스토랑을 매핑했습니다.")

try:
    df['date'] = pd.to_datetime(df['date'], unit='ms') # timestamp in milliseconds
except ValueError:
    df['date'] = pd.to_datetime(df['date']) # assume standard date string format if ms fails

df = df.sort_values(by='date').reset_index(drop=True)
total_reviews = len(df)
train_split = int(total_reviews * 0.7)
val_split_idx = int(total_reviews * (0.7 + 0.1)) # 검증 세트의 끝 인덱스
train_df = df.iloc[:train_split].copy()
val_df = df.iloc[train_split:val_split_idx].copy()
test_df = df.iloc[val_split_idx:].copy() # 나머지 20%를 테스트 세트로
print(f"학습 세트 크기: {len(train_df)}개 리뷰")
print(f"검증 세트 크기: {len(val_df)}개 리뷰")
print(f"테스트 세트 크기: {len(test_df)}개 리뷰")

print("\n--- 2단계: absa_atepc_results.json 파일에서 속성 데이터 로드 및 동적 어휘집 구축 시작 ---")

absa_data = {}
try:
    with open(absa_results_path, 'r', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)
            absa_data[entry['review_id']] = entry['aspects']
    print(f"'{absa_results_path}' 파일에서 {len(absa_data)}개의 리뷰 속성 데이터를 성공적으로 로드했습니다.")
except FileNotFoundError:
    print(f"오류: '{absa_results_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    sys.exit()
except json.JSONDecodeError as e:
    print(f"오류: '{absa_results_path}' 파일 파싱 중 오류 발생: {e}. 파일 내용이 올바른 JSON 형식을 따르는지 확인해주세요.")
    sys.exit()

all_extracted_aspect_terms_from_file = []
all_extracted_sentiments_from_file = []

for review_id in train_df['review_id']:
    aspects = absa_data.get(review_id, [])
    if not aspects: # 속성이 없는 경우 'general' 및 'neutral'로 대체
        all_extracted_aspect_terms_from_file.append('general')
        all_extracted_sentiments_from_file.append('neutral')
    else:
        for aspect in aspects:
            if 'term' in aspect and 'sentiment' in aspect:
                all_extracted_aspect_terms_from_file.append(aspect['term'].lower())
                # 감성 라벨을 소문자로 통일 (Positive, Negative, Neutral)
                all_extracted_sentiments_from_file.append(aspect['sentiment'].lower())

# 속성 키워드 어휘집 구축
term_counts = Counter(all_extracted_aspect_terms_from_file)
# <PAD>는 0번, <UNK>는 1번, 'general'은 2번 인덱스를 가집니다.
ASPECT_TO_ID = {'<PAD>': 0, '<UNK>': 1, 'general': 2}
next_id = 3
MIN_TERM_FREQUENCY = 1 # 어휘집 구축 시 최소 등장 빈도
filtered_terms = [term for term, count in term_counts.items() if count >= MIN_TERM_FREQUENCY]
filtered_terms.sort(key=lambda x: term_counts[x], reverse=True)

for term in filtered_terms:
    if term not in ASPECT_TO_ID:
        ASPECT_TO_ID[term] = next_id
        next_id += 1
NUM_ASPECT_TERMS = len(ASPECT_TO_ID)

# 감성 어휘집 구축 (고정된 긍정, 부정, 중립)
SENTIMENT_TO_ID = {'<PAD>': 0, '<UNK>': 1, 'positive': 2, 'negative': 3, 'neutral': 4}
NUM_SENTIMENTS = len(SENTIMENT_TO_ID)

print(f"동적으로 구축된 속성 어휘집 크기 (학습 데이터 기반): {NUM_ASPECT_TERMS}개 키워드")
print(f"어휘집 상위 10개 키워드: {list(ASPECT_TO_ID.items())[3:13]}...")
print(f"구축된 감성 어휘집 크기: {NUM_SENTIMENTS}개 감성 ({list(SENTIMENT_TO_ID.items())})")
print("absa_atepc_results.json 파일에서 속성 데이터 로드 및 동적 어휘집 구축 완료.")

print("\n--- 2.5단계: 로드된 속성 데이터를 DataFrame에 매핑 및 캐싱 시작 ---")

def map_aspect_and_sentiment_ids_from_file(df_subset, absa_data_map, aspect_to_id_map, sentiment_to_id_map):
    extracted_aspect_ids_list = []
    extracted_sentiment_ids_list = []
    
    for _, row in df_subset.iterrows():
        review_id = row['review_id']
        aspects_for_review = absa_data_map.get(review_id, [])
        
        aspect_ids_for_review = []
        sentiment_ids_for_review = []
        
        if aspects_for_review:
            for aspect in aspects_for_review:
                if 'term' in aspect and 'sentiment' in aspect:
                    aspect_id = aspect_to_id_map.get(aspect['term'].lower(), aspect_to_id_map['<UNK>'])
                    sentiment_id = sentiment_to_id_map.get(aspect['sentiment'].lower(), sentiment_to_id_map['<UNK>'])
                    
                    aspect_ids_for_review.append(aspect_id)
                    sentiment_ids_for_review.append(sentiment_id)
        
        if not aspect_ids_for_review:
            aspect_ids_for_review.append(aspect_to_id_map['general'])
            sentiment_ids_for_review.append(sentiment_to_id_map['neutral'])
            
        extracted_aspect_ids_list.append(aspect_ids_for_review)
        extracted_sentiment_ids_list.append(sentiment_ids_for_review)
        
    return extracted_aspect_ids_list, extracted_sentiment_ids_list

train_df['extracted_aspect_ids'], train_df['extracted_sentiment_ids'] = map_aspect_and_sentiment_ids_from_file(train_df, absa_data, ASPECT_TO_ID, SENTIMENT_TO_ID)
val_df['extracted_aspect_ids'], val_df['extracted_sentiment_ids'] = map_aspect_and_sentiment_ids_from_file(val_df, absa_data, ASPECT_TO_ID, SENTIMENT_TO_ID)
test_df['extracted_aspect_ids'], test_df['extracted_sentiment_ids'] = map_aspect_and_sentiment_ids_from_file(test_df, absa_data, ASPECT_TO_ID, SENTIMENT_TO_ID)

print("로드된 속성 데이터를 DataFrame에 매핑 및 캐싱 완료.")

class ReviewDataset(Dataset):
    def __init__(self, df):
        self.user_indices = torch.tensor(df['user_idx'].values, dtype=torch.long)
        self.business_indices = torch.tensor(df['business_idx'].values, dtype=torch.long)
        self.extracted_aspect_ids = df['extracted_aspect_ids'].values.tolist()
        self.extracted_sentiment_ids = df['extracted_sentiment_ids'].values.tolist()
        self.stars = torch.tensor(df['stars'].values, dtype=torch.float)
    def __len__(self):
        return len(self.stars)
    def __getitem__(self, idx):
        return (self.user_indices[idx], self.business_indices[idx],
                self.extracted_aspect_ids[idx], self.extracted_sentiment_ids[idx],
                self.stars[idx])

def custom_collate_fn(batch):
    user_indices, business_indices, extracted_aspect_ids_list, extracted_sentiment_ids_list, stars = zip(*batch)

    user_indices = torch.stack(user_indices)
    business_indices = torch.stack(business_indices)
    stars = torch.stack(stars)

    max_aspect_len = max(len(ids) for ids in extracted_aspect_ids_list)
    
    padded_aspect_ids = [ids + [ASPECT_TO_ID['<PAD>']] * (max_aspect_len - len(ids)) for ids in extracted_aspect_ids_list]
    padded_aspect_ids_tensor = torch.tensor(padded_aspect_ids, dtype=torch.long)
    
    padded_sentiment_ids = [ids + [SENTIMENT_TO_ID['<PAD>']] * (max_aspect_len - len(ids)) for ids in extracted_sentiment_ids_list]
    padded_sentiment_ids_tensor = torch.tensor(padded_sentiment_ids, dtype=torch.long)
    
    attn_mask = (padded_aspect_ids_tensor == ASPECT_TO_ID['<PAD>'])

    return user_indices, business_indices, padded_aspect_ids_tensor, padded_sentiment_ids_tensor, stars, attn_mask

print("\n--- 3단계: AAT-Rec 모델 구성 요소 정의 (감성 통합) ---")

class CustomerRestaurantInteractionModule(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim, mlp_dims, dropout_rate):
        super(CustomerRestaurantInteractionModule, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.business_embedding = nn.Embedding(num_businesses, embedding_dim)
        layers = []
        input_dim = embedding_dim * 2
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = input_dim

    def forward(self, user_indices, business_indices):
        user_emb = self.user_embedding(user_indices)
        business_emb = self.business_embedding(business_indices)
        combined_emb = torch.cat((user_emb, business_emb), dim=-1)
        interaction_features = self.mlp(combined_emb)
        return interaction_features

class ReviewAspectTermExtractionModule(nn.Module):
    def __init__(self, num_aspect_terms, aspect_embedding_dim, num_sentiments, sentiment_embedding_dim, num_heads, mlp_dims, dropout_rate):
        super(ReviewAspectTermExtractionModule, self).__init__()
        self.aspect_embedding = nn.Embedding(num_aspect_terms, aspect_embedding_dim, padding_idx=ASPECT_TO_ID['<PAD>'])
        self.sentiment_embedding = nn.Embedding(num_sentiments, sentiment_embedding_dim, padding_idx=SENTIMENT_TO_ID['<PAD>'])
        
        combined_embedding_dim = aspect_embedding_dim + sentiment_embedding_dim
        self.multihead_attn = nn.MultiheadAttention(embed_dim=combined_embedding_dim, num_heads=num_heads, batch_first=True)
        
        layers = []
        input_dim = combined_embedding_dim
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            if dropout_rate > 0: # 드롭아웃 레이어 조건부 추가
                layers.append(nn.Dropout(dropout_rate))
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = input_dim
            
    def forward(self, pre_extracted_aspect_ids_batch, pre_extracted_sentiment_ids_batch, attn_mask):
        embedded_terms = self.aspect_embedding(pre_extracted_aspect_ids_batch)
        embedded_sentiments = self.sentiment_embedding(pre_extracted_sentiment_ids_batch)
        
        combined_embeddings = torch.cat((embedded_terms, embedded_sentiments), dim=-1)
        
        attn_output, _ = self.multihead_attn(
            combined_embeddings, 
            combined_embeddings, 
            combined_embeddings, 
            key_padding_mask=attn_mask
        )

        valid_attn_output = attn_output * (~attn_mask.unsqueeze(-1)).float()
        pooled_output = valid_attn_output.sum(dim=1) / (~attn_mask).sum(dim=1, keepdim=True).float().clamp(min=1)
        
        aspect_features = self.mlp(pooled_output)
        return aspect_features

class RatingPredictionModule(nn.Module):
    def __init__(self, input_dim, mlp_dims, dropout_rate):
        super(RatingPredictionModule, self).__init__()
        layers = []
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            if dropout_rate > 0: # 드롭아웃 레이어 조건부 추가
                layers.append(nn.Dropout(dropout_rate))
            input_dim = dim
        layers.append(nn.Linear(input_dim, 1))
        self.mlp = nn.Sequential(*layers)

    def forward(self, combined_features):
        raw_prediction = self.mlp(combined_features)
        prediction = torch.sigmoid(raw_prediction) * 4 + 1
        return prediction

class AATRec(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim,
                 user_biz_mlp_dims, num_aspect_terms, aspect_embedding_dim,
                 num_sentiments, sentiment_embedding_dim,
                 num_attn_heads, aspect_mlp_dims, final_mlp_dims, dropout_rate):
        super(AATRec, self).__init__()
        self.customer_restaurant_module = CustomerRestaurantInteractionModule(
            num_users, num_businesses, embedding_dim, user_biz_mlp_dims, dropout_rate
        )
        self.aspect_extraction_module = ReviewAspectTermExtractionModule(
            num_aspect_terms, aspect_embedding_dim, num_sentiments, sentiment_embedding_dim,
            num_attn_heads, aspect_mlp_dims, dropout_rate
        )
        combined_feature_dim = self.customer_restaurant_module.output_dim + \
                               self.aspect_extraction_module.output_dim
        self.rating_prediction_module = RatingPredictionModule(
            combined_feature_dim, final_mlp_dims, dropout_rate
        )
    def forward(self, user_indices, business_indices, pre_extracted_aspect_ids, pre_extracted_sentiment_ids, attn_mask):
        interaction_features = self.customer_restaurant_module(user_indices, business_indices)
        aspect_features = self.aspect_extraction_module(pre_extracted_aspect_ids, pre_extracted_sentiment_ids, attn_mask)
        combined_features = torch.cat((interaction_features, aspect_features), dim=-1)
        predicted_rating = self.rating_prediction_module(combined_features).squeeze()
        return predicted_rating

print("AATRec 모델 구성 요소 정의 완료.")


# --- 4. 하이퍼파라미터 설정 및 그리드 서치 수행 ---

print("\n--- 4단계: 그리드 서치를 통한 모델 학습 및 저장 시작 ---")

train_dataset = ReviewDataset(train_df)
val_dataset = ReviewDataset(val_df)
test_dataset = ReviewDataset(test_df)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"학습 장치: '{device}'")

# --- 고정된 하이퍼파라미터 (명시된 대로) ---
FIXED_EMBEDDING_DIM = 64
FIXED_LEARNING_RATE = 1e-4
FIXED_BATCH_SIZE = 128
FIXED_FINAL_MLP_DIMS = [32, 16] # 2개의 은닉층 + 1개 출력층 = 총 3개 선형 레이어
FIXED_DROPOUT_RATE = 0.2 # 드롭아웃 고정 
FIXED_NUM_ATTN_HEADS = 8 # 헤드 수 고정
# ---------------------------------------------------

# 그리드 서치할 나머지 하이퍼파라미터 공간 정의
param_grid = {
    'aspect_embedding_dim': [64, 96, 128], # 속성 키워드 임베딩 차원
    'sentiment_embedding_dim': [16, 32, 48], # 감성 임베딩 차원 (총합 고려하여 조절)
    'user_biz_mlp_dims': [[128, 64]], # [256, 128] 제거
    'aspect_mlp_dims': [[64, 32]], # [128, 64] 제거
}

best_overall_rmse = float('inf')
best_params = None
best_model_path = 'best_aatrec_model_grid_search_final_fixed_params.pt' # 파일명 변경

# 하이퍼파라미터 조합 생성
keys = param_grid.keys()
combinations = itertools.product(*param_grid.values())
total_combinations = len(list(itertools.product(*param_grid.values())))
print(f"총 {total_combinations}개의 하이퍼파라미터 조합을 탐색합니다.")

for i, values in enumerate(combinations):
    current_params = dict(zip(keys, values))
    
    # 고정된 파라미터 추가
    current_params['embedding_dim'] = FIXED_EMBEDDING_DIM
    current_params['learning_rate'] = FIXED_LEARNING_RATE
    current_params['batch_size'] = FIXED_BATCH_SIZE
    current_params['final_mlp_dims'] = FIXED_FINAL_MLP_DIMS 
    current_params['dropout_rate'] = FIXED_DROPOUT_RATE 
    current_params['num_attn_heads'] = FIXED_NUM_ATTN_HEADS 

    print(f"\n--- 조합 {i+1}/{total_combinations} 학습 시작 ---")
    print(f"현재 파라미터: {current_params}")

    EMBEDDING_DIM = current_params['embedding_dim']
    ASPECT_EMBEDDING_DIM = current_params['aspect_embedding_dim']
    SENTIMENT_EMBEDDING_DIM = current_params['sentiment_embedding_dim']
    NUM_ATTN_HEADS = current_params['num_attn_heads']
    LEARNING_RATE = current_params['learning_rate']
    BATCH_SIZE = current_params['batch_size']
    DROPOUT_RATE = current_params['dropout_rate']
    USER_BIZ_MLP_DIMS = current_params['user_biz_mlp_dims']
    ASPECT_MLP_DIMS = current_params['aspect_mlp_dims']
    FINAL_MLP_DIMS = current_params['final_mlp_dims']

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate_fn)

    model = AATRec(
        num_users, num_businesses, EMBEDDING_DIM,
        USER_BIZ_MLP_DIMS, NUM_ASPECT_TERMS, ASPECT_EMBEDDING_DIM,
        NUM_SENTIMENTS, SENTIMENT_EMBEDDING_DIM,
        NUM_ATTN_HEADS, ASPECT_MLP_DIMS, FINAL_MLP_DIMS, DROPOUT_RATE
    )
    model.to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    PATIENCE = 5
    MIN_DELTA = 0.001
    current_val_rmse_for_combo = float('inf') 
    epochs_no_improve = 0
    NUM_EPOCHS_PER_COMBINATION = 30 

    for epoch in range(NUM_EPOCHS_PER_COMBINATION):
        model.train()
        total_train_loss = 0
        for user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars, attn_mask in train_loader:
            user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars = \
                user_indices.to(device), business_indices.to(device), extracted_aspect_ids.to(device), extracted_sentiment_ids.to(device), stars.to(device)
            attn_mask = attn_mask.to(device)
            
            optimizer.zero_grad()
            predicted_stars = model(user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, attn_mask)
            loss = criterion(predicted_stars, stars)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        avg_train_loss = total_train_loss / len(train_loader)

        model.eval()
        total_val_loss = 0
        val_preds = []
        val_true = []
        with torch.no_grad():
            for user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars, attn_mask in val_loader:
                user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars = \
                    user_indices.to(device), business_indices.to(device), extracted_aspect_ids.to(device), extracted_sentiment_ids.to(device), stars.to(device)
                attn_mask = attn_mask.to(device)
                predicted_stars = model(user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, attn_mask)
                total_val_loss += criterion(predicted_stars, stars).item()
                val_preds.extend(predicted_stars.cpu().numpy())
                val_true.extend(stars.cpu().numpy())
        avg_val_loss = total_val_loss / len(val_loader)
        
        val_preds_clipped = np.clip(val_preds, 1.0, 5.0)
        current_val_rmse = np.sqrt(mean_squared_error(val_true, val_preds_clipped))

        sys.stdout.write(f"\r    Epoch {epoch+1}/{NUM_EPOCHS_PER_COMBINATION}, "
                         f"Train Loss: {avg_train_loss:.4f}, "
                         f"Val Loss: {avg_val_loss:.4f}, "
                         f"Val RMSE: {current_val_rmse:.4f}")
        sys.stdout.flush()

        if current_val_rmse < current_val_rmse_for_combo - MIN_DELTA:
            current_val_rmse_for_combo = current_val_rmse
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            
        if current_val_rmse < best_overall_rmse:
            best_overall_rmse = current_val_rmse
            best_params = current_params.copy() 
            torch.save(model.state_dict(), best_model_path)
            sys.stdout.write(f" --> 전체 최적 RMSE 개선됨: {best_overall_rmse:.4f}. 모델 저장됨.\n")
            sys.stdout.flush()

        if epochs_no_improve == PATIENCE:
            sys.stdout.write(f"\n    조기 종료! {PATIENCE} 에포크 동안 검증 RMSE 개선이 없었습니다 (현재 조합).\n")
            sys.stdout.flush()
            break

print("\n--- 그리드 서치 완료 ---")
print(f"최종 최적 RMSE: {best_overall_rmse:.4f}")
print(f"최적 하이퍼파라미터: {best_params}")
print(f"최적 모델 저장 경로: {best_model_path}")


# --- 최종 모델 테스트 ---
print("\n--- 최종 모델 테스트 시작 (그리드 서치를 통해 찾은 최적 파라미터 사용) ---")

if os.path.exists(best_model_path) and best_params:
    final_model = AATRec(
        num_users, num_businesses, best_params['embedding_dim'],
        best_params['user_biz_mlp_dims'], NUM_ASPECT_TERMS, best_params['aspect_embedding_dim'],
        NUM_SENTIMENTS, best_params['sentiment_embedding_dim'],
        best_params['num_attn_heads'], best_params['aspect_mlp_dims'], best_params['final_mlp_dims'], best_params['dropout_rate']
    )
    final_model.load_state_dict(torch.load(best_model_path, map_location=device))
    final_model.to(device)
    print(f"최적의 모델 가중치 '{best_model_path}' 로드 완료.")
else:
    print("경고: 최적의 모델 가중치를 찾을 수 없거나 최적 파라미터가 설정되지 않았습니다. 테스트를 건너뜜.")
    sys.exit()

test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'], shuffle=False, collate_fn=custom_collate_fn)

final_model.eval()
test_preds = []
test_true = []
with torch.no_grad():
    for user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars, attn_mask in test_loader:
        user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars = \
            user_indices.to(device), business_indices.to(device), extracted_aspect_ids.to(device), extracted_sentiment_ids.to(device), stars.to(device)
        attn_mask = attn_mask.to(device)
        predicted_stars = final_model(user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, attn_mask)
        test_preds.extend(predicted_stars.cpu().numpy())
        test_true.extend(stars.cpu().numpy())

test_preds_clipped = np.clip(test_preds, 1.0, 5.0)

mse = mean_squared_error(test_true, test_preds_clipped)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_true, test_preds_clipped)
mape = mean_absolute_percentage_error(test_true, test_preds_clipped)

print(f"\n--- 최종 모델 성능 평가 (그리드 서치를 통해 찾은 최적 파라미터) ---")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"사용된 최적 하이퍼파라미터: {best_params}")

--- 1단계: 데이터 로드 및 전처리 시작 ---
'447796'개의 리뷰 데이터를 성공적으로 로드했습니다.
'27807'명의 고유 사용자와 '6831'개의 고유 레스토랑을 매핑했습니다.
학습 세트 크기: 313457개 리뷰
검증 세트 크기: 44779개 리뷰
테스트 세트 크기: 89560개 리뷰

--- 2단계: absa_atepc_results.json 파일에서 속성 데이터 로드 및 동적 어휘집 구축 시작 ---
'absa_atepc_results.json' 파일에서 447796개의 리뷰 속성 데이터를 성공적으로 로드했습니다.
동적으로 구축된 속성 어휘집 크기 (학습 데이터 기반): 39160개 키워드
어휘집 상위 10개 키워드: [('food', 3), ('service', 4), ('staff', 5), ('place', 6), ('atmosphere', 7), ('prices', 8), ('they', 9), ('price', 10), ('pizza', 11), ('it', 12)]...
구축된 감성 어휘집 크기: 5개 감성 ([('<PAD>', 0), ('<UNK>', 1), ('positive', 2), ('negative', 3), ('neutral', 4)])
absa_atepc_results.json 파일에서 속성 데이터 로드 및 동적 어휘집 구축 완료.

--- 2.5단계: 로드된 속성 데이터를 DataFrame에 매핑 및 캐싱 시작 ---
로드된 속성 데이터를 DataFrame에 매핑 및 캐싱 완료.

--- 3단계: AAT-Rec 모델 구성 요소 정의 (감성 통합) ---
AATRec 모델 구성 요소 정의 완료.

--- 4단계: 그리드 서치를 통한 모델 학습 및 저장 시작 ---
학습 장치: 'cuda'
총 9개의 하이퍼파라미터 조합을 탐색합니다.

--- 조합 1/9 학습 시작 ---
현재 파라미터: {'aspect_embedding_dim': 64, 'sentiment_embedding_dim': 16, 'user_biz_mlp_